In [ ]:
# Importing libraries
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from IPython.display import display, HTML

from scipy.constants import c, G, astronomical_unit as AU

In [ ]:
# Plot settings
plt.rc("font", size=10)
plt.rcParams['font.family'] = 'Liberation Serif'

In [ ]:
# Constants
M_sun = 1.9884e30  # kg

# Mercury parameters
M_mercury = 0.3301e24  # kg
T_mercury = 87.969  # days
semi_major_axis = 57.909e9  # meters
eccentricity = 0.2056

In [ ]:
# Calculating perehelion distans and initial velocity at perihelion using Kepler's equation
perihelion_distance = semi_major_axis * (1 - eccentricity)

print(f"perihelion_distance = {perihelion_distance:.2e} m")

v_perihelion = np.sqrt(G * M_sun * (1 + eccentricity) / (semi_major_axis * (1 - eccentricity)))

print(f"v_perihelion = {v_perihelion:.2e} m/s")

# Calculating base acceleration at perihelion
a_perihelion = G * M_sun / perihelion_distance**2

print(f"a_perihelion = {a_perihelion:.2e} m/s^2")

In [ ]:
# Time parameters
T = 100 * T_mercury # iteractions in days
dt = (2 * v_perihelion / a_perihelion) / 86400 / 20 # days

steps = int(T / dt)

print(f"steps = {steps}")

In [ ]:
# Initial conditions
r0 = np.array([perihelion_distance, 0, 0])  # Mercury starts at perihelion
v0 = np.array([0, v_perihelion, 0])  # tangential velocity

In [ ]:
def acceleration(r, alpha, beta):
    r_mag = np.linalg.norm(r)  # magnitude of the position vector
    factor = -G * M_sun / r_mag**3 * (1 + alpha * G * M_sun / (c**2 * r_mag) + beta * (G * M_sun / (c**2 * r_mag))**2)
    return factor * r  # acceleration vector

def simulate_orbit(alpha, beta):
    r = np.zeros((steps, 3))
    v = np.zeros((steps, 3))
    r[0] = r0
    v[0] = v0
    
    for i in range(steps - 1):
        a = acceleration(r[i], alpha, beta)
        v_half = v[i] + 0.5 * a * dt * 86400  # days to seconds
        r[i+1] = r[i] + v_half * dt * 86400
        a_new = acceleration(r[i+1], alpha, beta)
        v[i+1] = v_half + 0.5 * a_new * dt * 86400
    
    return r

In [ ]:
# Runing simulation 
alpha, beta = 0, 10**6  

r = simulate_orbit(alpha, beta)

In [ ]:
# Real-time animation
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlabel('X (AU)')
ax.yaxis.label.set_fontsize(20)
ax.set_ylabel('Y (AU)')
ax.xaxis.label.set_fontsize(20)
ax.tick_params(axis="both", which="major", length=10, width=0.5, labelsize=15)
ax.tick_params(axis="both", which="minor", length=5, width=0.5, labelsize=15)
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.5, 0.5)
ax.set_aspect('equal')

ax.scatter(0, 0, c='yellow', marker='o', s=200, label='Sun')

trajectory, = ax.plot([], [], 'r', linewidth=0.2, label='Mercury orbit')
mercury_dot, = ax.plot([], [], 'bo', markersize=5, label='Mercury')

def init():
    trajectory.set_data([], [])
    mercury_dot.set_data([], [])
    return trajectory, mercury_dot

def update(frame):
    if frame >= len(r):
        return trajectory, mercury_dot
    trajectory.set_data(r[:frame+1, 0] / AU, r[:frame+1, 1] / AU)
    mercury_dot.set_data([r[frame, 0] / AU], [r[frame, 1] / AU])
    return trajectory, mercury_dot

ani = animation.FuncAnimation(fig, update, frames=len(r), init_func=init, interval=100, blit=True)
plt.legend()
plt.tight_layout()

display(HTML(ani.to_jshtml()))

In [ ]:
# Static plot
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlabel('X (AU)')
ax.yaxis.label.set_fontsize(20)
ax.set_ylabel('Y (AU)')
ax.xaxis.label.set_fontsize(20)
ax.tick_params(axis="both", which="major", length=10, width=0.5, labelsize=15)
ax.tick_params(axis="both", which="minor", length=5, width=0.5, labelsize=15)
 

ax.plot(r[:, 0] / AU, r[:, 1] / AU, 'r', linewidth=0.2, label='Mercury orbit')
ax.scatter(0, 0, c='yellow', marker='o', s=200, label='Sun')

ax.legend()

plt.tight_layout() 
plt.savefig('images/orbit_simulation.png', bbox_inches='tight')
plt.show()